In [1]:
import numpy as np
import copy

import heapq

from collections import defaultdict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt


In [2]:
with open('input_day_16.txt', 'r') as file:
    input_string = file.read()


test_input_string = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
""".strip()

test_input_string = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
""".strip()


In [3]:
grid = input_string.split("\n")

R = len(grid)
C = len(grid[0])

start_r, start_c = (R-2, 1)
end_r, end_c = (1, C-2)

def estimate_cost(state):

    dir, pos, cost = state
    pos_r, pos_c = pos

    dir_cost = 2000
    if dir == "E" and pos_r == end_r:
        dir_cost = 0
    elif dir == "N" and pos_c == end_c:
        dir_cost = 0
    elif dir == "N":
        dir_cost = 1000
    elif dir == "E":
        dir_cost = 1000

    return cost + dir_cost + pos_r - end_r + pos_c - end_c

best_cost = 1000000000

q = []

heapq.heapify(q)

start_state = ("E", (R-2, 1), 0)

heapq.heappush(q, (estimate_cost(start_state), start_state))

positions_checked = set()

num_states_considered = 0

while len(q) > 0:
    est_cost, state = heapq.heappop(q)
    dir, pos, cost = state
    pos_r, pos_c = pos

    positions_checked.add(pos)

    #if num_states_considered % 10_000 == 0:
    #    print(num_states_considered, state)
    #num_states_considered += 1

    if pos_r == end_r and pos_c == end_c:
        best_cost = min(best_cost, cost)

    for new_dir in "ESNW":

        if dir == "N" and new_dir == "S": continue
        if dir == "S" and new_dir == "N": continue
        if dir == "W" and new_dir == "E": continue
        if dir == "E" and new_dir == "W": continue
        
        additional_cost = 1001
        if dir == new_dir:
            additional_cost = 1

        if new_dir == "E":
            new_r = pos_r
            new_c = pos_c + 1
        elif new_dir == "W":
            new_r = pos_r
            new_c = pos_c - 1
        elif new_dir == "N":
            new_r = pos_r - 1
            new_c = pos_c
        elif new_dir == "S":
            new_r = pos_r + 1
            new_c = pos_c

        if grid[new_r][new_c] == "." or grid[new_r][new_c] == "E":
            new_state = (new_dir, (new_r, new_c), cost + additional_cost)
            if (new_r, new_c) not in positions_checked:
                heapq.heappush(q, (estimate_cost(new_state), new_state))

print("part 1:", best_cost)

part 1: 83444


In [4]:
def estimate_cost(state):

    dir, pos, cost, prev_set = state
    pos_r, pos_c = pos

    dir_cost = 2000
    if dir == "E" and pos_r == end_r:
        dir_cost = 0
    elif dir == "N" and pos_c == end_c:
        dir_cost = 0
    elif dir == "N":
        dir_cost = 1000
    elif dir == "E":
        dir_cost = 1000

    return cost + dir_cost + pos_r - end_r + pos_c - end_c

visited_states = set()

best_costs = {}
best_costs_pos = {}
start_set = set()
start_set.add((R-2, 1))
start_state = ("E", (R-2, 1), 0, start_set)

combined_end_set = set()


frontier_states = []
heapq.heapify(frontier_states)
heapq.heappush(frontier_states, (estimate_cost(start_state), start_state))

best_costs[("E", (R-2, 1))] = 0
best_costs_pos[(R-2, 1)] = 0

num_states_considered = 0
while len(frontier_states) > 0:

    pair = heapq.heappop(frontier_states)

    if num_states_considered % 10_000 == 0:
        print(num_states_considered, state[:-1], len(frontier_states), len(best_costs), len(best_costs_pos))

    num_states_considered += 1

    est_cost, state = pair

    dir, pos, cost, prev_set = state
    pos_r, pos_c = pos

    if est_cost > best_cost:
        continue

    for new_dir in "ESNW":

        if dir == "N" and new_dir == "S": continue
        if dir == "S" and new_dir == "N": continue
        if dir == "W" and new_dir == "E": continue
        if dir == "E" and new_dir == "W": continue
        
        additional_cost = 1001
        if dir == new_dir:
            additional_cost = 1

        if new_dir == "E":
            new_r = pos_r
            new_c = pos_c + 1
        elif new_dir == "W":
            new_r = pos_r
            new_c = pos_c - 1
        elif new_dir == "N":
            new_r = pos_r - 1
            new_c = pos_c
        elif new_dir == "S":
            new_r = pos_r + 1
            new_c = pos_c

        if not (grid[new_r][new_c] == "." or grid[new_r][new_c] == "E"):
            continue

        if (new_r, new_c) in best_costs_pos:
            prev_best_cost_pos = best_costs_pos[(new_r, new_c)]
            if cost + additional_cost > prev_best_cost_pos + 1000:
                continue

        if (new_dir, (new_r, new_c)) in best_costs:
            prev_best_cost = best_costs[(new_dir, (new_r, new_c))]

            if cost + additional_cost > prev_best_cost:
                continue

            if cost + additional_cost < prev_best_cost:
                new_set = copy.deepcopy(prev_set)
                new_set.add((new_r, new_c))
                new_state = (new_dir, (new_r, new_c), cost + additional_cost, new_set)
                if new_state not in frontier_states:
                    heapq.heappush(frontier_states, (estimate_cost(new_state), new_state))
                    best_costs[(new_dir, (new_r, new_c))] = cost + additional_cost
                    best_costs_pos[(new_r, new_c)] = cost + additional_cost

            if cost + additional_cost == prev_best_cost:
                new_set = copy.deepcopy(prev_set)
                new_set.add((new_r, new_c))
                combined_set = new_set.union(prev_set)
                new_state = (new_dir, (new_r, new_c), cost + additional_cost, combined_set)
                if len(combined_set) > len(prev_set):
                    if new_state not in frontier_states:
                        heapq.heappush(frontier_states, (estimate_cost(new_state), new_state))
                        best_costs[(new_dir, (new_r, new_c))] = cost + additional_cost
                        best_costs_pos[(new_r, new_c)] = cost + additional_cost

        else: # first visit
            new_set = copy.deepcopy(prev_set)
            new_set.add((new_r, new_c))
            new_state = (new_dir, (new_r, new_c), cost + additional_cost, new_set)
            if new_state not in frontier_states:
                heapq.heappush(frontier_states, (estimate_cost(new_state), new_state))
                best_costs[(new_dir, (new_r, new_c))] = cost + additional_cost
                best_costs_pos[(new_r, new_c)] = cost + additional_cost

    if pos == (end_r, end_c):
        if cost == best_cost:
            #print("best solution!", cost)
            combined_end_set = combined_end_set.union(prev_set)
        else:
            pass
            #print("solution!", cost)


print("part 2:", len(combined_end_set))


0 ('W', (61, 91)) 0 1 1
10000 ('E', (63, 5), 28116) 176 2530 2271
20000 ('W', (27, 6), 40171) 347 4751 4268
30000 ('W', (1, 14), 50209) 260 6530 5914
40000 ('N', (46, 125), 63349) 476 8011 7303
50000 ('E', (27, 121), 68380) 693 8827 8005
60000 ('E', (27, 130), 72393) 1005 9426 8579
70000 ('S', (38, 107), 75405) 799 9867 8969
80000 ('W', (35, 100), 78417) 1307 10269 9357
90000 ('E', (27, 92), 82435) 1203 10648 9704
part 2: 483
